# Interfacing C libraries

This is a very short introduction into how you can use external DLLs from Python.

Out example includes a simple primality testing library that we write in standard C.

In [27]:
!cat primelib/primelib.h

﻿#ifndef PRIMELIB_H
#define PRIMELIB_H

#if defined(_MSC_VER)
    //  Microsoft 
    #define EXPORT __declspec(dllexport)
    #define IMPORT __declspec(dllimport)
#elif defined(__GNUC__)
    //  GCC
    #define EXPORT __attribute__((visibility("default")))
    #define IMPORT
#else
    //  do nothing and hope for the best?
    #define EXPORT
    #define IMPORT
    #pragma warning Unknown dynamic link import/export semantics.
#endif

EXPORT int is_prime(unsigned long long x);
EXPORT unsigned long long smallest_divisor(unsigned long long x);

#endif


In [28]:
!cat primelib/primelib.c

﻿#include "primelib.h"

#include "math.h"

EXPORT int is_prime(unsigned long long x)
{
	if (x == 1) return 0;
	return smallest_divisor(x) == x;
}

EXPORT unsigned long long smallest_divisor(unsigned long long x)
{
	if (x < 4) return x;
	unsigned long long max_divisor = (unsigned long long)sqrt((float)x) + 1;
	for (unsigned long long i = 2; i < max_divisor; i++) {
		if (x % i == 0) return i;
	}
	return x;
}


## [ctypes](https://docs.python.org/3/library/ctypes.html)

`ctypes` is a foreign function library for Python. It provides C compatible data types, and allows calling functions in DLLs or shared libraries. It can be used to wrap these libraries in pure Python. (cited)

### In Windows

In [53]:
from ctypes import windll

primelib = windll.LoadLibrary(".\\primelib.dll")  # Find the proper path.
primelib.is_prime


In [4]:
primelib.is_prime(2)

1

Cool!

In [7]:
primelib.is_prime(22801763489)

ArgumentError: argument 1: <class 'OverflowError'>: int too long to convert

Not so cool. But it should work, ehm...

By default, all functions are considered to accept integer parameters and return
integer results (convention from C). We have a different type, so let's specify
this explicitly:

In [58]:
from ctypes import c_ulonglong, c_bool

primelib.is_prime.argtypes = [c_ulonglong]
primelib.is_prime.restype = c_bool  # Note: bool IS int in C, only we convert it in Python
primelib.smallest_divisor.argtypes = [c_ulonglong]
primelib.smallest_divisor.restype = c_ulonglong

{x : primelib.smallest_divisor(x) for x in range(10**12, 10**12+20)}

{1000000000000: 2,
 1000000000001: 73,
 1000000000002: 2,
 1000000000003: 61,
 1000000000004: 2,
 1000000000005: 3,
 1000000000006: 2,
 1000000000007: 34519,
 1000000000008: 2,
 1000000000009: 29,
 1000000000010: 2,
 1000000000011: 3,
 1000000000012: 2,
 1000000000013: 7,
 1000000000014: 2,
 1000000000015: 5,
 1000000000016: 2,
 1000000000017: 3,
 1000000000018: 2,
 1000000000019: 1601}

In [57]:
primelib.is_prime(2)

True

Note that for several system libraries, the interface is already prepared by ctypes.

In [47]:
# In Windows
mkdir = windll.kernel32.CreateDirectoryW
mkdir("Created_from_WinAPI", 0)

0

In [51]:
!ls -l | grep "Created*"

drwxr-xr-x 1 janpi 197609       0 Mar  1 14:39 Created_from_WinAPI


### In Linux

In [3]:
from ctypes import cdll

primelib = cdll.LoadLibrary("./libprimelib.so")

In [4]:
from ctypes import c_ulonglong

primelib.is_prime.argtypes = [c_ulonglong]
primelib.smallest_divisor.argtypes = [c_ulonglong]
primelib.smallest_divisor.restype = c_ulonglong

In [6]:
{x : primelib.smallest_divisor(x) for x in range(10**12, 10**12+20)}

{1000000000000: 2,
 1000000000001: 73,
 1000000000002: 2,
 1000000000003: 61,
 1000000000004: 2,
 1000000000005: 3,
 1000000000006: 2,
 1000000000007: 34519,
 1000000000008: 2,
 1000000000009: 29,
 1000000000010: 2,
 1000000000011: 3,
 1000000000012: 2,
 1000000000013: 7,
 1000000000014: 2,
 1000000000015: 5,
 1000000000016: 2,
 1000000000017: 3,
 1000000000018: 2,
 1000000000019: 1601}

## [cffi](https://cffi.readthedocs.io/en/latest/)

CFFI is an external package providing a C Foreign Function Interface for Python. CFFI allows one to interact with almost any C code from Python. However, C++ is not currently supported. User needs to add C-like declarations to Python code and, even though the declarations can often be directly copy-pasted from C headers or documentation, some understanding of C is normally required.

CFFI has two different main modes, “ABI” and “API”. In ABI mode one accesses the library at binary level, while in API mode a separate compilation step with a C compiler is used. ABI mode can be easier to start with, but API mode is faster and more robust and is thus normally the recommended mode.

We will wrap the same library here: (Note that in Windows, the `*.lib` files must be found in the same place as the `*.dll`.)

### API style

In [16]:
import cffi

ffi = cffi.FFI()

ffi.cdef("""
    int is_prime(unsigned long long);
    unsigned long long smallest_divisor(unsigned long long);
""")

ffi.set_source("_primelib_cffi",
"""
     #include "primelib/primelib.h"
""",
    libraries=['primelib'],
    library_dirs =["."]
)

ffi.compile(verbose=True)

generating ./_primelib_cffi.c
(already up-to-date)
the current directory is '/home/honza/hilase-python-course-2021'
running build_ext
building '_primelib_cffi' extension
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/home/honza/hilase-python-course-2021/.venv/include -I/usr/include/python3.8 -c _primelib_cffi.c -o ./_primelib_cffi.o
x86_64-linux-gnu-gcc -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 ./_primelib_cffi.o -L. -lprimelib -o ./_primelib_cffi.cpython-38-x86_64-linux-gnu.so


'/home/honza/hilase-python-course-2021/_primelib_cffi.cpython-38-x86_64-linux-gnu.so'

In [26]:
from _primelib_cffi.lib import is_prime, smallest_divisor

In [31]:
smallest_divisor(44864561566508441)

7161929

### ABI style (in-line)

In [70]:
ffi = cffi.FFI()

ffi.cdef("""
    int is_prime(unsigned long long);
    unsigned long long smallest_divisor(unsigned long long);
""")

lib = ffi.dlopen(".\\primelib.dll")
lib.smallest_divisor(455484446111)


367